# LangGraph Tutorial: Tools Creation

**Objective:** Build custom financial calculation tools that extend LLM capabilities.

**Tools to Build:**
1. Currency Converter (USD, EUR, GBP, INR, JPY)
2. EMI Calculator (loan payment calculator)

## Setup

Import dependencies and load environment.

In [ ]:
# Core imports
from langchain_core.tools import tool

In [ ]:
@tool
def greet(name: str) -> str:
    """Greet a person by name"""
    return f"Hello, {name}! Welcome to our financial assistant."

In [ ]:
# this will fail as the function is defined as langgraph tool
greet("Donald")

In [ ]:
# this is how we invoke the langgraph tool
greet.invoke({"name": "Donald"})

## Tool 1: Currency Converter

**What it does:** Converts amounts between USD, EUR, GBP, INR, JPY

**Key points:**
- `@tool` decorator makes it LLM-callable
- Docstring helps LLM understand when to use it
- Type hints define parameter types
- Returns human-readable strings

In [ ]:
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Args:
        amount: The amount to convert
        from_currency: Source currency code (USD, EUR, GBP, INR, JPY)
        to_currency: Target currency code (USD, EUR, GBP, INR, JPY)
    
    Returns:
        A string with the conversion result including the exchange rate
    """
    
    # Simplified exchange rates (relative to USD)
    # In production, fetch real-time rates from an API
    exchange_rates = {
        "USD": 1.0,
        "EUR": 0.92,
        "GBP": 0.79,
        "INR": 83.12,
        "JPY": 149.50
    }
    
    # Validate currencies
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}. Supported: {', '.join(exchange_rates.keys())}"
    
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}. Supported: {', '.join(exchange_rates.keys())}"
    
    # Convert to USD first, then to target currency
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    
    # Calculate the effective exchange rate
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    result = (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )
    
    return result

print("✅ currency_converter tool defined")
print(f"   Type: {type(currency_converter)}")
print(f"   Name: {currency_converter.name}")

## Test Currency Converter

Test the tool directly before integrating with the graph.

In [ ]:
print("=" * 70)
print("TESTING: Currency Converter")
print("=" * 70)

# Test Case 1: Standard conversion
print("\n[Test 1] Convert 1000 USD to EUR")
result = currency_converter.invoke({"amount": 1000, "from_currency": "USD", "to_currency": "EUR"})
print(result)

# Test Case 2: Error handling - invalid currency
print("\n[Test 2] Invalid currency (XYZ)")
result = currency_converter.invoke({"amount": 100, "from_currency": "XYZ", "to_currency": "USD"})
print(result)

# Test Case 3: Same currency conversion
print("\n[Test 3] Convert INR to INR (edge case)")
result = currency_converter.invoke({"amount": 5000, "from_currency": "INR", "to_currency": "INR"})
print(result)

print("\n" + "=" * 70)
print("✅ All tests passed")

## Tool 2: EMI Calculator

**What it does:** Calculates monthly loan payments (EMI = Equated Monthly Installment)

**Formula:** `EMI = P × r × (1 + r)^n / [(1 + r)^n - 1]`
- P = Principal (loan amount)
- r = Monthly interest rate (annual rate / 12 / 100)
- n = Tenure in months

In [ ]:
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Args:
        principal: The loan amount (principal)
        annual_interest_rate: Annual interest rate as a percentage (e.g., 8.5 for 8.5%)
        tenure_months: Loan tenure in months
        currency: Currency code for display (USD, EUR, GBP, INR, JPY)
    
    Returns:
        A string with the EMI calculation details
    """
    
    # Validate inputs
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    # Convert annual interest rate to monthly rate (decimal)
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    # Handle zero interest rate edge case
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        # EMI formula: P × r × (1 + r)^n / [(1 + r)^n - 1]
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    result = (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months ({tenure_months // 12} years, {tenure_months % 12} months)\n"
        f"  \n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )
    
    return result

print("✅ emi_calculator tool defined")
print(f"   Type: {type(emi_calculator)}")
print(f"   Name: {emi_calculator.name}")

## Test EMI Calculator

In [ ]:
print("=" * 70)
print("TESTING: EMI Calculator")
print("=" * 70)

# Test Case 1: Standard loan
print("\n[Test 1] Car loan: 500,000 INR at 8.5% for 5 years")
result = emi_calculator.invoke({
    "principal": 500000,
    "annual_interest_rate": 8.5,
    "tenure_months": 60,
    "currency": "INR"
})
print(result)

# Test Case 2: Home loan
print("\n[Test 2] Home loan: 300,000 USD at 6.5% for 30 years")
result = emi_calculator.invoke({
    "principal": 300000,
    "annual_interest_rate": 6.5,
    "tenure_months": 360,
    "currency": "USD"
})
print(result)

# Test Case 3: Zero interest (edge case)
print("\n[Test 3] Interest-free loan: 10,000 EUR for 12 months")
result = emi_calculator.invoke({
    "principal": 10000,
    "annual_interest_rate": 0,
    "tenure_months": 12,
    "currency": "EUR"
})
print(result)

# Test Case 4: Error handling - invalid principal
print("\n[Test 4] Invalid principal (0)")
result = emi_calculator.invoke({
    "principal": 0,
    "annual_interest_rate": 8.5,
    "tenure_months": 24,
    "currency": "USD"
})
print(result)

print("\n" + "=" * 70)
print("✅ All tests passed")

## Verify Tool Schemas

Check the JSON schema that LLMs use to understand these tools.

In [ ]:
from pprint import pprint

In [ ]:
print("Currency Converter Schema:")
print("=" * 70)
cc_schema = currency_converter.args_schema.model_json_schema()
pprint(cc_schema)

In [ ]:
print("\n\nEMI Calculator Schema:")
print("=" * 70)
ec_schema = emi_calculator.args_schema.model_json_schema()
pprint(ec_schema)

---

## ✅ Tools Created!

**Built:**
- ✅ Currency Converter (5 currencies)
- ✅ EMI Calculator (with edge case handling)

**Verified:**
- ✅ Tools work correctly in isolation
- ✅ Error handling functions properly
- ✅ JSON schemas are well-formed

**Next:** Notebook 3 - Graph Construction (connecting tools to LLM)